### 기존 머신러닝 방식
- EEGNet 모델 사용 전, SEED dataset을 이용하여 tutorial에 있던 코드로 SEED Dataset(라벨링 2: 긍정, 1: 중립, 0: 부정 )을 Train/test 8:2로 split 하여 Train 및 Test 진행.
- Original dataset shape: (50910, 5, 62)(b)전처리 과정에서 LDA로 차원 축소 진행
- 0.77의 나쁘지 않은 정확도 도출. 특히 Positive를 분류하는데 0.87의 수치로 돋보이는 결과를 확인할 수 있다.
- 하지만 15명의 뇌파를 모두 섞여서 Data Leakage 나 Overfitting 가능성 존재


In [45]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.metrics import classification_report, accuracy_score

# Load data
labels = np.load('../assets/seed-dataset/LabelsNoImage.npz')['arr_0']  # Assuming 'arr_0' contains labels
subjects = np.load('../assets/seed-dataset/SubjectsNoImage.npz')['arr_0']  # Assuming 'arr_0' contains subject data
dataset = np.load('../assets/seed-dataset/DatasetCaricatoNoImage.npz')['arr_0']  # Assuming 'arr_0' contains features


# Check the shape of the dataset
print(f"Original dataset shape: {dataset.shape}")

# If the dataset is 3D, reshape it to 2D
if len(dataset.shape) == 3:
    dataset = dataset.reshape(dataset.shape[0], -1)  # Flatten the dataset

print(f"Reshaped dataset shape: {dataset.shape}")

# Step 1: Split data into training and testing
X_train, X_test, y_train, y_test = train_test_split(dataset, labels, test_size=0.2, random_state=42)

# Step 2: Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Step 3: Apply PCA for dimensionality reduction
pca = PCA(n_components=0.95)  # Preserve 95% of variance
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

# Step 4: Apply LDA for classification
lda = LDA()
lda.fit(X_train_pca, y_train)
y_pred = lda.predict(X_test_pca)

# Step 5: Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, target_names=['Negative', 'Neutral', 'Positive'])



print(f'Accuracy: {accuracy:.4f}')
print('Classification Report:')
print(report)


Original dataset shape: (50910, 5, 62)
Reshaped dataset shape: (50910, 310)
Accuracy: 0.7728
Classification Report:
              precision    recall  f1-score   support

    Negative       0.71      0.74      0.72      3301
     Neutral       0.74      0.76      0.75      3372
    Positive       0.87      0.82      0.85      3509

    accuracy                           0.77     10182
   macro avg       0.77      0.77      0.77     10182
weighted avg       0.78      0.77      0.77     10182



### LDA 기반 피험자 독립 검증 코드

- 기존 LDA 기반 검증에서 과적합 되었는지 탐지해보기 위해 피험자 독립 검증으로 진행. 
- Subject[0](0번 피험자)를 테스트용으로 지정하고, 나머지 서브젝트(1번~14번 피험자)로 Train 
- 피험자 0 독립 테스트 정확도: 0.3326
- 이전의 정확성은 신뢰성이 없는 지표임을 확인 -> Braindecode의 EEGNet모델을 이용하여 

In [70]:
# 1. 특정 피험자(0번)를 테스트용으로 지정
test_subject_id = 0 

# 2. 인덱스 분리
train_idx = np.where(subjects == test_subject_id, False, True) 
test_idx = np.where(subjects == test_subject_id, True, False)


# 3. 데이터 분할 및 2차원 변환 (Reshape)
# 여기서 .reshape(len(...), -1)을 통해 (샘플, 5*62) 형태로 확실히 만듭니다.
X_train = dataset[train_idx].reshape(np.sum(train_idx), -1)
y_train = labels[train_idx]

X_test = dataset[test_idx].reshape(np.sum(test_idx), -1)
y_test = labels[test_idx]

print(f"X_train 형태: {X_train.shape}") # (47516, 310) 처럼 2차원이어야 함
print(f"X_test 형태: {X_test.shape}")   # (3394, 310) 처럼 2차원이어야 함

# 4. Standaradization 
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 5. PCA 및 LDA 진행
pca = PCA(n_components=0.95)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

lda = LDA()
lda.fit(X_train_pca, y_train)
y_pred = lda.predict(X_test_pca)

# 결과 출력
print(f"피험자 {test_subject_id} 독립 테스트 정확도: {accuracy_score(y_test, y_pred):.4f}")

[False False False ...  True  True  True] [ True  True  True ... False False False]
X_train 형태: (47516, 310)
X_test 형태: (3394, 310)
피험자 0 독립 테스트 정확도: 0.3326


In [46]:
# 1. 테스트 세트에서 긍정(2), 중립(1), 부정(0)인 데이터 5개씩만 뽑기

pos_indices = np.where(y_test == 2)[0][:5]
neu_indices = np.where(y_test == 1)[0][:5]
neg_indices = np.where(y_test == 0)[0][:5]

test_indices = np.concatenate([pos_indices, neu_indices, neg_indices])

if len(test_indices) > 0:
    X_sample = X_test[test_indices] # 이미 스케일링 된 X_test 사용
    # PCA는 이미 적용된 X_test_pca에서 가져오는 게 더 정확함
    X_sample_pca = X_test_pca[test_indices]
    
    # 2. 예측 수행
    sample_preds = lda.predict(X_sample_pca)
    
    print("실제 정답:", y_test[test_indices])
    print("모델 예측:", sample_preds)
else:
    print("데이터셋에 1이나 2 라벨이 하나도 없습니다! y_test를 다시 확인하세요.")

실제 정답: [2 2 2 2 2 1 1 1 1 1 0 0 0 0 0]
모델 예측: [0 2 2 2 2 1 0 1 1 1 0 0 2 0 2]


In [56]:
import torch
from braindecode.models import EEGNetv4
from skorch.classifier import NeuralNetClassifier
from skorch.callbacks import EpochScoring
import numpy as np
# 3차원 기존 데이터 로드
labels = np.load('../assets/seed-dataset/LabelsNoImage.npz')['arr_0']  # Assuming 'arr_0' contains labels
subjects = np.load('../assets/seed-dataset/SubjectsNoImage.npz')['arr_0']  # Assuming 'arr_0' contains subject data
dataset = np.load('../assets/seed-dataset/DatasetCaricatoNoImage.npz')['arr_0']  # Assuming 'arr_0' contains features

# 1. 데이터 형태 변경 (Batch, Channels, Time)
# SEED DE features: (50910, 5, 62) -> (50910, 62, 5)
X_braindecode = dataset.transpose(0, 2, 1).astype(np.float32)
y_braindecode = labels.astype(np.int64)

# 2. 피험자 독립 분할 (Subject-Independent Split) - 아까처럼 0번 피험자 제외
test_subject_id = 0
train_idx = np.where(subjects != test_subject_id)[0]
test_idx = np.where(subjects == test_subject_id)[0]

X_train, X_test = X_braindecode[train_idx], X_braindecode[test_idx]
y_train, y_test = y_braindecode[train_idx], y_braindecode[test_idx]

print(f"입력 데이터 형태: {X_train.shape}") # (47516, 62, 5)


입력 데이터 형태: (47516, 62, 5)
(3394, 62, 5)


In [65]:
# 4. Skorch 기반 분류기 생성
clf = NeuralNetClassifier(
    model,
    criterion=torch.nn.CrossEntropyLoss,
    optimizer=torch.optim.Adam,
    train_split=None, 
    optimizer__lr=0.001,
    batch_size=64, # 데이터가 충분하므로 64는 적절합니다.
    max_epochs=20,
    device=device,
    # [수정] 에러를 유발하는 콜백을 잠시 제거하고 기본 로그만 확인합니다.
    callbacks=None 
)

# 5. 데이터 타입 강제 재지정 (중요!)
# PyTorch는 float32(X)와 long(y) 타입을 엄격하게 요구합니다.
X_train = X_train.astype(np.float32)
y_train = y_train.astype(np.int64)

# 6. 학습 실행
clf.fit(X_train, y_train)

  epoch    train_loss      dur
-------  ------------  -------
      1        1.0247  42.2323
      2        0.9722  11.5750
      3        0.9345  14.8737
      4        0.9287  9.6127
      5        0.9160  9.0090
      6        0.9014  10.7427
      7        0.8939  8.6988
      8        0.8837  35.7124
      9        0.8739  19.2816
     10        0.8707  9.3139
     11        0.8673  11.5837
     12        0.8588  8.6699
     13        0.8359  8.4512
     14        0.8161  18.7314
     15        0.8207  9.3711
     16        0.8088  30.2360
     17        0.7943  23.4470
     18        0.8051  16.1267
     19        0.7898  9.1643
     20        0.7828  9.3499


,module,EEGNetv4( (...') ) ) )
,criterion,<class 'torch...sEntropyLoss'>
,train_split,None
,classes,None
,optimizer,<class 'torch...im.adam.Adam'>
,lr,0.01
,max_epochs,20
,batch_size,64
,iterator_train,<class 'torch...r.DataLoader'>
,iterator_valid,<class 'torch...r.DataLoader'>
,dataset,<class 'skorc...aset.Dataset'>


In [66]:
from sklearn.metrics import classification_report, accuracy_score

# 1. 테스트 데이터 예측
y_pred = clf.predict(X_test)

# 2. 성능 출력
final_acc = accuracy_score(y_test, y_pred)
print(f"--- Braindecode EEGNet Evaluation ---")
print(f"Subject {test_subject_id} Accuracy: {final_acc:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=['Negative', 'Neutral', 'Positive']))

--- Braindecode EEGNet Evaluation ---
Subject 0 Accuracy: 0.4723

Classification Report:
              precision    recall  f1-score   support

    Negative       0.32      0.35      0.33      1120
     Neutral       1.00      0.04      0.08      1104
    Positive       0.55      1.00      0.71      1170

    accuracy                           0.47      3394
   macro avg       0.62      0.46      0.37      3394
weighted avg       0.62      0.47      0.38      3394



In [67]:
import numpy as np
from sklearn.metrics import accuracy_score, classification_report

# 1. 0번 피험자 데이터 분할 (시간 순서대로 자르는 것이 가장 정석입니다)
# 앞의 200개 샘플은 '교정(Calibration)'용, 나머지는 '진짜 테스트'용
n_calibration = 200
X_calib = X_test[:n_calibration]
y_calib = y_test[:n_calibration]

X_final_test = X_test[n_calibration:]
y_final_test = y_test[n_calibration:]

# 2. 기존 모델의 학습 설정 변경 (매우 세밀하게 조정)
# 학습률을 낮추고 에폭을 짧게 가져가서 기존의 지식을 잃지 않게 합니다.
clf.set_params(max_epochs=10, optimizer__lr=0.0001)

# 3. partial_fit으로 0번 피험자에게 맞춤형 학습 진행
print("개인화 학습(Fine-tuning) 진행 중...")
clf.partial_fit(X_calib, y_calib)

# 4. 최종 성능 평가
y_final_pred = clf.predict(X_final_test)
final_acc = accuracy_score(y_final_test, y_final_pred)

print("-" * 30)
print(f"개인화(Calibration) 적용 후 최종 정확도: {final_acc:.4f}")
print("-" * 30)
print(classification_report(y_final_test, y_final_pred, target_names=['Negative', 'Neutral', 'Positive']))

개인화 학습(Fine-tuning) 진행 중...
     21        0.0130  0.9938
     22        0.0143  0.3626
     23        0.0177  0.1721
     24        0.0170  0.3932
     25        0.0223  0.1649
     26        0.0175  0.0896
     27        0.0141  0.0544
     28        0.0167  0.0607
     29        0.0147  0.0641
     30        0.0187  0.0549
------------------------------
개인화(Calibration) 적용 후 최종 정확도: 0.6080
------------------------------
              precision    recall  f1-score   support

    Negative       0.56      0.44      0.49      1120
     Neutral       0.73      0.43      0.55      1104
    Positive       0.59      1.00      0.74       970

    accuracy                           0.61      3194
   macro avg       0.63      0.62      0.59      3194
weighted avg       0.63      0.61      0.59      3194

